In [1]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands


import glob 
import pickle
import numpy as np


def get_jpg_files_list():
    jpg_files_list = glob.glob("*.jpg")
    return jpg_files_list



def save_obj(obj, name ):
    with open('objects/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        

def load_obj(name ):
    with open('objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    

In [2]:
# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
#cap = cv2.VideoCapture(0)


file_name = "./Victory.MP4"
#file_name = "./Ok_.MP4"
#file_name = "./WTF.MP4"

cap = cv2.VideoCapture(file_name)  #MAH04285.MP4

label = None
if file_name[2:5] =="Vic":
    label = 0
if file_name[2:5] =="Ok_":
    label = 1
if file_name[2:5] =="WTF":
    label = 2


data_list=[]
count=0
with mp_hands.Hands(
    min_detection_confidence=0.75,
    min_tracking_confidence=0.5,
    max_num_hands=1) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue
        
        palm_vector_list = []
        
        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        count=count+1
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = hands.process(image)

        # Draw the face mesh annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                #print(len(results.multi_hand_landmarks[0].landmark))
                for n_point in range(len(results.multi_hand_landmarks[0].landmark)):
                        palm_vector_list.append(hand_landmarks.landmark[n_point].x)
                        palm_vector_list.append(hand_landmarks.landmark[n_point].y)                        
                        palm_vector_list.append(hand_landmarks.landmark[n_point].z)                
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS) 
        
            #если собираем датасет то добавим метку класса
            palm_vector_list.append(label)
            data_list.append(np.array(palm_vector_list))
            
            #save_obj(results.multi_hand_landmarks, './video/landmarks_palm__' + str(label) +"___"+ str(count) )   
            cv2.putText(image, 
                    'emotion', (100, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

            cv2.imshow('MediaPipe PalmMesh', image)
            #cv2.imwrite('video/palm_' +str(label)  +"___" + str(count) + '.png', image)

            
        if cv2.waitKey(5) & 0xFF == 27:
                break



d=np.array(data_list)
save_obj(d, './video/data__' + str(label) )   
cap.release()
cv2.destroyAllWindows()